# <font color = 'red'>Clustering algorithme</font>

In [24]:
!pip install kmodes

In [25]:
import numpy as np
import pandas as pd
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Upload the data

In [26]:
from google.colab import files
uploaded = files.upload()

Saving Book1.xlsx to Book1 (1).xlsx


In [ ]:
import pandas as pd
data = pd.read_excel('Book1.xlsx', index_col=0, header=0)
data.head()

# Data preprocessing

In [ ]:
def time(row):
    return datetime.strptime(row, '%H:%M')

def minutes(row):
    return row.total_seconds() / (60*60)
data['date_diff_hour'] = (data['Date_entree'] - data['Date_sortie'] + data['Heure_entree'].apply(time) - data['Heure_sortie'].apply(time)).apply(minutes)

In [ ]:
def covid(row):
    if row < datetime.strptime('2020/03/16', '%Y/%m/%d'):
        return 'Avant_covid'
    else :
        return 'Pendant_covid'
data['Covid'] = data['Date_sortie'].apply(covid)


In [ ]:
def time(row):
    if datetime.strptime(row, '%H:%M') <= datetime.strptime('12:00', '%H:%M'):
        return 'Matin'
    elif datetime.strptime(row, '%H:%M') > datetime.strptime('12:00', '%H:%M') and datetime.strptime(row, '%H:%M') <= datetime.strptime('18:00', '%H:%M') : 
        return 'Apres-midi'
    elif datetime.strptime(row, '%H:%M') > datetime.strptime('18:00', '%H:%M'): 
        return 'Nuit'
data['Heure_entree'] = data['Heure_entree'].apply(time)
data['Heure_sortie'] = data['Heure_sortie'].apply(time)

In [ ]:
data = data.drop(['Date_entree','Date_sortie','Num_REMORQUE'],axis = 1)

In [ ]:
scaler = MinMaxScaler()
data[['Poids_charge',	'Poids_vide','date_diff_hour']] = scaler.fit_transform(data[['Poids_charge',	'Poids_vide','date_diff_hour']])

# Final state of the data

In [ ]:
data.head()

In [ ]:
data_values = data.values
data_values[:,3] = data_values[:,3].astype(float) 
data_values[:,4] = data_values[:,4].astype(float) 
data_values[:,8] = data_values[:,8].astype(float) 

# Finding the optimum value for the number of clusters (k) using the elbow method

In [ ]:
Cost_proto = []
for i in range(1,11):
  kproto = KPrototypes(n_clusters=i,max_iter=20,verbose=0).fit(data_values, categorical = [0,1,2,5,6,7,9])
  Cost_proto.append(kproto.cost_)

In [ ]:
plt.plot(range(1,11), Cost_proto, marker = 'o')
plt.xlabel('Number of clusters')
plt.ylabel('Cost')
plt.show()

# We choose k =5

In [ ]:
kproto = KPrototypes(n_clusters=5,max_iter=200,verbose=0)
clusters = kproto.fit_predict(data_values, categorical = [0,1,2,5,6,7,9])

# Result

In [ ]:
data['clusters'] = clusters
data.head()

# Comparison with the k-means method that uses only numerical data

# Finding the optimum value for the number of clusters (k) using the elbow method

In [ ]:
X = data[['Poids_charge',	'Poids_vide','date_diff_hour']].values
cost = []
for i in range(1,11):
  kmeans = KMeans(n_clusters=i).fit(X)
  cost.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1,11), cost, marker = 'o')
plt.xlabel('Number of clusters')
plt.ylabel('Cost')
plt.show()

# We choose k = 4

In [ ]:
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(X)

In [ ]:
data['clusters'] = clusters
data.head()